<div class="alert alert-block alert-info" style="background-color:">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is an example of <b>mounting virtual media for HPE iLO systems</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

#from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success"><h2 style="font-size:40px">2.</h2>This is our mount_virtual_media_iso function overview, in the end this will give us with virtual media related information.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to obtain the virtual media information.

1. In the <b>mount_virtual_media_iso</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. In the ILO rest api the information about virtual media is stored inside the <b>managers json object</b>.
    Through managers_response variable  we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET operation</a> to and trying to obtain the virtual media about iLO system.
    <br>

3. In the managers response object we are iterating on all the members with different <b>odata_id</b> .
    Then again an http GET request is made and response is stored in <b>managers_members_response</b>.
    The virtual media information is ultimately present in the <b>managers_members_response</b>.
    <br>

4. Else if resource directory is enabled then we will simply iterate through all instances inside resources 
    and if virtual media is present in <b>@odata.type</b> attribute then we will store information about virtual media in virtual_media_uri and break out of iteration.
    <br>

Ultimately if virtual_media_uri was obtained without any errors then get the information at the relevant uri and print out that information .

 </div>

In [ ]:
def mount_virtual_media_iso(_redfishobj, media_type,):

    virtual_media_uri = None
    virtual_media_response = []

    resource_instances = _redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        virtual_media_uri = managers_members_response.obj['VirtualMedia']['@odata.id']
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#VirtualMediaCollection.' in instance['@odata.type']:
                virtual_media_uri = instance['@odata.id']

    if virtual_media_uri:
        virtual_media_response = _redfishobj.get(virtual_media_uri)
        for virtual_media_slot in virtual_media_response.obj['Members']:
            data = _redfishobj.get(virtual_media_slot['@odata.id'])
            if media_type in data.dict['MediaTypes']:
                virtual_media_mount_uri = data.obj['Actions']['#VirtualMedia.EjectMedia']['target']
                post_body = {"Action": "HpeiLOVirtualMedia.EjectVirtualMedia"}
                resp = _redfishobj.post(virtual_media_mount_uri, post_body)
                if resp.status == 400:
                    try:
                        print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                         sort_keys=True))
                    except Exception as excp:
                        sys.stderr.write("A response error occurred, unable to access iLO"
                                         "Extended Message Info...")
                elif resp.status != 200:
                    sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
                else:
                    print("Success!\n")
                    print(json.dumps(resp.dict, indent=4, sort_keys=True))
                break

<div class="alert alert-block alert-success"><h2 style="font-size:40px">3.</h2>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
        <li>Once your credentials that are enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>If not then we make a call to <b>mount_virtual_media_iso</b> to obtain virtual media related info of system.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #specify the type of content the media represents
    MEDIA_TYPE = "CD" #current possible options: Floppy, USBStick, CD, DVD

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                   password=LOGIN_PASSWORD)

        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    mount_virtual_media_iso(REDFISHOBJ, MEDIA_TYPE)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-warning"><h2 style="font-size:40px">4.</h2>
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success"><h2 style="font-size:40px">5.</h2>In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>